# Latent Guassian coupla model for mixed data
Let $X_1$ be continuous and $X_2$ be truncated.

In [1]:
import sys
sys.path.append('..')
from SCCA_new import *
import pickle
import pandas as pd


In [2]:
import os
os.getcwd()

'/projectnb/labci/Qiuyun/SCCA/v2/mixdata'

In [22]:
folder = "100-100_200_0.8_0.9/"

In [23]:
vx = pd.read_csv('../continuous/data/'+ folder+'vx.csv', header = None).values
vy = pd.read_csv('../continuous/data/'+ folder+'vy.csv', header = None).values
lam = 0.8
p1 = len(vx)
p2 = len(vy)
d = p1 + p2
SigX = pd.read_csv('../continuous/data/'+ folder+'SigX.csv', header = None).values
SigY = pd.read_csv('../continuous/data/'+ folder+'SigX.csv', header = None).values
SigXY = lam * SigX.dot(vx).dot(vy.T).dot(SigY)
Sigma = np.concatenate((np.concatenate((SigX, SigXY), axis=1),
                        np.concatenate((SigXY.T, SigY), axis=1)), axis = 0)

A = np.vstack((np.hstack((np.zeros(shape = (p1, p1)), SigXY))
                    ,np.hstack((SigXY.T, np.zeros(shape = (p2, p2))))))

B = np.vstack((np.hstack((SigX, np.zeros(shape = (p1, p2))))
                    ,np.hstack((np.zeros(shape = (p2, p1)), SigY))))
nonzero = sum(vx!=0)

# Experiement

In [33]:
num_data = 100
truncate = 1
sigmasq = 0.2
rho_0 = 10
Niter = 10000
step = 0.1
results = np.zeros(shape = (num_data, 5))
res_folder = 'result/' + folder + 'NormInitm_ST_%.1f_%.1f_%.1f_%f/' %(truncate, sigmasq, rho_0, step)
isExist = os.path.exists(res_folder)
Ifplot = False

In [34]:
def cal_error(x, vx):
    return min((norm(x / norm(x) - vx/norm(vx))**2), (norm(x/ norm(x) + vx/norm(vx))**2))

In [ ]:
result = {}
delta_all = {}
theta_all = {}
I_all = {}

if not isExist:
  # Create a new directory because it does not exist 
    os.makedirs(res_folder)
    print("The new directory is created!")
    

for C in [-2, -1, 0]:
    result[C] = np.zeros((num_data, 8))
    delta_all[C] = np.zeros((num_data, Niter-1, d))
    theta_all[C] = np.zeros((num_data, Niter-1, d))
    I_all[C] = np.zeros((num_data, Niter-1))

for nrun in range(num_data):
    Z =  pd.read_csv('../continuous/data/'+ folder+'data%i.csv' %nrun, header = None).values
    print(nrun)
    for C in [-2, -1,  0]:
        # Generate data
        X = Z[:, 0:d//2]
        Y = Z[:,d//2:]
        n = len(Z)
        sparsity = np.sum(Y<C)/ n/d*2
        init = pd.read_csv('data/'+ folder+'minit_PD%i_%d.csv' %(nrun, C), header = None).values[:,0]
        init = init / np.std(init)
        Rhat = pd.read_csv('data/'+ folder+'R_PD%i_%d.csv' %(nrun, C), header = 0, index_col= 0).values
        Sighat_X = Rhat[:d//2, :d//2]
        Sighat_Y = Rhat[d//2:, d//2:]
        Sighat_XY = Rhat[:d//2, d//2:]
        Ahat = np.vstack((np.hstack((np.zeros(shape = (d//2, d//2)), Sighat_XY))
                                        ,np.hstack((Sighat_XY.T, np.zeros(shape = (d//2, d//2))))))
        Bhat = np.concatenate((np.concatenate((Sighat_X, np.zeros(shape = (d//2, d//2))), axis=0),
                                        np.concatenate((np.zeros(shape = (d//2, d//2)), Sighat_Y), axis=0)), axis = 1) / (n*2)
        now1 = datetime.now()
        error, Res_theta, Res_delta, swap_atem, swap, mAcc, loggam, I, phi,aa, visits = Simu_t(Ahat, Bhat, vx[:, 0]/ norm(vx), vy[:, 0]/ norm(vy), d, n,A,B, temp = np.arange(1,0.5, -0.1)
                                                , sigmasq = sigmasq, initial=[init, np.ones(d)], Niter = Niter, update = True, rho_0= rho_0, 
                                                                                           loggam = np.zeros(5) -4, truncate = truncate, step = step)
        result[C][nrun, 7] = (datetime.now() - now1).total_seconds()
        delta_all[C][nrun] = Res_delta
        theta_all[C][nrun] = Res_theta
        I_all[C][nrun] = I

        ll = Res_delta[I==0].shape[0]
        burnin = ll//4*3
        delta_hat = stats.mode(Res_delta[burnin:, :])[0][0]
        # sparsity level
        result[C][nrun, 0 ] = sparsity
        # squared l2 error, TPR & RNR
        vxhat = Res_theta[I ==0][burnin:, :p1] * delta_hat[:p1]
        vyhat = Res_theta[I ==0][burnin:, p1:] *delta_hat[p1:]
        result[C][nrun, 1] = cal_error(np.mean(vxhat, axis = 0)*delta_hat[:p1], vx[:, 0]) #norm(np.mean(vxhat, axis = 0)*delta_hat[:p1] - vx.dot(vx.T)/norm(vx)**2, ‘fro’)
        result[C][nrun, 2] = cal_error(np.mean(vyhat, axis = 0)*delta_hat[p1:], vy[:, 0])
        result[C][nrun, 3] = np.sum(delta_hat[:p1][ vx[:, 0] != 0]) /nonzero
        result[C][nrun, 4] = np.sum(delta_hat[p1:][ vy[:, 0] != 0]) /nonzero
        result[C][nrun, 5] = 1 - np.sum(delta_hat[:p1][vx[:, 0] == 0]) /(p1 -nonzero)
        result[C][nrun, 6] = 1 - np.sum(delta_hat[p1:][vy[:, 0] == 0]) /(p2 -nonzero)
        if Ifplot:
            fig, ax = plt.subplots(4, 1, sharey=False, tight_layout=True , figsize = (10,10))

            ax[0].plot( error[I==0,2], 'b.', alpha=0.5 )
            ax[0].set_xlabel("Iteration")
            ax[0].set_ylabel(r"$error$")
            ax[0].set_title(r"error for $v_x$ when p=%i & N=%i" % (d, n))
            ax[0].grid()

            ax[1].plot( error[I==0,3], 'b.', alpha=0.5 )
            ax[1].set_xlabel("Iteration")
            ax[1].set_ylabel(r"$error$")
            ax[1].set_title(r"error for $v_y$ when p=%i & N=%i" % (d, n))
            ax[1].grid()

            ax[2].plot(Res_theta[I==0, 0]) 
            ax[2].set_title(r'$\theta$')

            ## eigenvalue
            ax[3].plot(error[I==0,1], 'b.')
            ax[3].set_title(r"eigenvalues")
            ax[3].grid()
            plt.show()
            plt.plot(delta_hat)
            plt.show()
        print((nrun, C), result[C][nrun])
            


In [ ]:
with open(res_folder + 'ST_rhat_error.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(result, f)

In [ ]:
with open(res_folder + 'ST_rhat_error.pkl', 'rb') as f:  # Python 3: open(..., 'wb')
    result1 =  pickle.load(f)
i = 0
for key in result1:
    me = np.mean(result1[key], axis = 0)
    sd = np.std(result1[key], axis = 0)
    print(key, '& %.2f & %.2f (%.2f) & %.2f (%.2f) \\\\' %(me[0], me[1],sd[1], me[2], sd[2]))
    i = i +1 

In [27]:
result = {}
delta_all = {}
theta_all = {}
I_all = {}
aaa = {}
bbb = {}
if not isExist:
  # Create a new directory because it does not exist 
    os.makedirs(res_folder)
    print("The new directory is created!")
    
Niter = 2000
for C in [-2, -1, 0]:
    result[C] = np.zeros((num_data, 9))
    I_all[C] = np.zeros((num_data, Niter))
    a = result1[C][:, 1:3]
    aaa[C] = np.mean(a[a<1])
    b = result1[C][:, 2]
    bbb[C] = np.mean(b[b<1])

for nrun in range(num_data):
    Z =  pd.read_csv('../continuous/data/'+ folder+'data%i.csv' %nrun, header = None).values
    print(nrun)
    for C in [-2, -1,  0]:
        # Generate data
        X = Z[:, 0:d//2]
        Y = Z[:,d//2:]
        n = len(Z)
        sparsity = np.sum(Y<C)/ n/d*2
        init = pd.read_csv('data/'+ folder+'minit%i_%d.csv' %(nrun, C), header = None).values[:,0]
        init = init / np.std(init)
        Rhat = pd.read_csv('data/'+ folder+'R%i_%d.csv' %(nrun, C), header = 0, index_col= 0).values
        Sighat_X = Rhat[:d//2, :d//2]
        Sighat_Y = Rhat[d//2:, d//2:]
        Sighat_XY = Rhat[:d//2, d//2:]
        Ahat = np.vstack((np.hstack((np.zeros(shape = (d//2, d//2)), Sighat_XY))
                                        ,np.hstack((Sighat_XY.T, np.zeros(shape = (d//2, d//2))))))
        Bhat = np.concatenate((np.concatenate((Sighat_X, np.zeros(shape = (d//2, d//2))), axis=0),
                                        np.concatenate((np.zeros(shape = (d//2, d//2)), Sighat_Y), axis=0)), axis = 1) / (n*2)
        now1 = datetime.now()
        limit = np.mean(result1[C], axis = 0)
        error, Res_theta, Res_delta, swap_atem, swap, mAcc, loggam, I, phi,aa, visits = Simu_t(Ahat, Bhat, vx[:, 0]/ norm(vx), vy[:, 0]/ norm(vy), d, n,A,B, temp = np.arange(1,0.5, -0.1)
                                                , sigmasq = sigmasq, initial=[init, np.ones(d)], Niter = Niter, update = True, rho_0= rho_0, 
                                                                                           loggam = np.zeros(5) -4, truncate = truncate, step = step, tol = 0.0001, err = [aaa[C], bbb[C]])
        result[C][nrun, 7] = (datetime.now() - now1).total_seconds()
        result[C][nrun, 8] = len(Res_theta)

        ll = Res_delta[I==0].shape[0]
        burnin = ll//4*3
        delta_hat = stats.mode(Res_delta)[0][0]
        # sparsity level
        result[C][nrun, 0 ] = sparsity
        # squared l2 error, TPR & RNR
        vxhat = Res_theta[I ==0][burnin:, :p1] * delta_hat[:p1]
        vyhat = Res_theta[I ==0][burnin:, p1:] *delta_hat[p1:]
#         result[C][nrun, 1] = cal_error(np.mean(vxhat, axis = 0)*delta_hat[:p1], vx[:, 0])
#         result[C][nrun, 2] = cal_error(np.mean(vyhat, axis = 0)*delta_hat[p1:], vy[:, 0])
        result[C][nrun, 1] = cal_error(np.mean(vxhat, axis = 0)*delta_hat[:p1], vx[:, 0])
        result[C][nrun, 2] = cal_error(np.mean(vyhat, axis = 0)*delta_hat[p1:], vy[:, 0])
        result[C][nrun, 3] = np.sum(delta_hat[:p1][ vx[:, 0] != 0]) /3
        result[C][nrun, 4] = np.sum(delta_hat[p1:][ vy[:, 0] != 0]) /3
        result[C][nrun, 5] = 1 - np.sum(delta_hat[:p1][vx[:, 0] == 0]) /(p1 -3)
        result[C][nrun, 6] = 1 - np.sum(delta_hat[p1:][vy[:, 0] == 0]) /(p2 -3)
        print(C, result[C][nrun, :])


0
-2 [1.95333333e-02 5.08963977e-02 6.85010270e-01 1.00000000e+00
 6.66666667e-01 9.89690722e-01 9.69072165e-01 3.07113300e+00
 2.24000000e+02]
-1 [1.46800000e-01 2.65550778e-02 4.32405068e-01 1.00000000e+00
 6.66666667e-01 9.89690722e-01 9.89690722e-01 3.49608400e+00
 2.66000000e+02]
0 [4.78666667e-01 9.96325135e-02 1.10801106e+00 1.00000000e+00
 6.66666667e-01 9.89690722e-01 9.79381443e-01 2.58755280e+01
 1.99900000e+03]
1
-2 [2.59333333e-02 1.32580858e+00 6.89494138e-01 3.33333333e-01
 6.66666667e-01 9.48453608e-01 9.48453608e-01 1.29282510e+01
 9.48000000e+02]
-1 [1.62933333e-01 4.66669556e-01 2.08541961e-01 1.00000000e+00
 1.00000000e+00 9.27835052e-01 9.27835052e-01 2.67970300e+00
 1.88000000e+02]
0 [4.93533333e-01 2.00000000e+00 1.26697413e+00 0.00000000e+00
 3.33333333e-01 9.38144330e-01 9.48453608e-01 2.69081480e+01
 1.99900000e+03]
2
-2 [2.20666667e-02 2.00000000e+00 2.00000000e+00 0.00000000e+00
 0.00000000e+00 9.58762887e-01 9.17525773e-01 2.70448460e+01
 1.99900000e+03]
-1

In [35]:
with open(res_folder + 'ST_computation_error.pkl', 'rb') as f:  # Python 3: open(..., 'wb')
    result= pickle.load( f)

In [36]:

i = 0
for key in result:
    me = np.mean(result[key], axis = 0)
    sd = np.std(result[key], axis = 0)
    print(key, '& %.2f & %.2f (%.2f) & %.2f (%.2f) \\\\' %(me[0],me[7],sd[7], me[8],sd[8]))
    i = i +1 

-2 & 0.02 & 25.46 (29.60) & 1742.11 (2010.35) \\
-1 & 0.16 & 27.29 (28.72) & 1884.25 (1977.84) \\
0 & 0.49 & 41.24 (31.44) & 2865.63 (2178.06) \\
